In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Tried to automate, but didn't work :(
# import os

# repo_root = os.path.join(os.path.dirname(os.getcwd()))
# !export PYTHONPATH={repo_root}

In [3]:
import sys
from os.path import join

try:
    from clip_grounding.utils.paths import REPO_PATH
    sys.path.append(join(REPO_PATH, "CLIP_explainability/Transformer-MM-Explainability/"))
except ImportError:
    print("Cannot import 'clip_grounding.utils.paths.REPO_PATH'")
    print("To fix: use 'export PYTHONPATH=$PWD'")

In [4]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch
import CLIP.clip as clip
import cv2
from PIL import Image
from glob import glob
from natsort import natsorted

from clip_grounding.utils.paths import REPO_PATH
from clip_grounding.utils.io import load_json
from clip_grounding.utils.visualize import set_latex_fonts, show_grid_of_images
from clip_grounding.utils.image import pad_to_square
from clip_grounding.datasets.png_utils import show_images_and_caption
from clip_grounding.datasets.png import (
    PNG,
    visualize_item,
    overlay_segmask_on_image,
    overlay_relevance_map_on_image,
    get_text_colors,
)
from clip_grounding.evaluation.clip_on_png import (
    process_entry_image_to_text,
    process_entry_text_to_image,
    interpret_and_generate,
)

## Load dataset

In [5]:
dataset = PNG(dataset_root=join(REPO_PATH, "data/panoptic_narrative_grounding"), split="val2017")

## Load model

In [6]:
# load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

### Load and visualize a sample

In [7]:
def visualize_entry_text_to_image(entry, pad_images=True, figsize=(18, 5)):
    test_img, test_texts, orig_image = process_entry_text_to_image(entry, unimodal=False)
    outputs = interpret_and_generate(model, test_img, test_texts, orig_image, return_outputs=True, show=False)
    relevance_map = outputs[0]["image_relevance"]
    
    image_with_mask = overlay_segmask_on_image(entry["image"], entry["image_mask"])
    if pad_images:
        image_with_mask = pad_to_square(image_with_mask)
    
    image_with_relevance_map = overlay_relevance_map_on_image(entry["image"], relevance_map)
    if pad_images:
        image_with_relevance_map = pad_to_square(image_with_relevance_map)
    
    text_colors = get_text_colors(entry["text"], entry["text_mask"])
    
    show_images_and_caption(
        [image_with_mask, image_with_relevance_map],
        entry["text"], text_colors, figsize=figsize,
        image_xlabels=["Ground truth segmentation", "Predicted relevance map"]
    )

In [9]:
cv2.__version__

'4.5.3'

In [8]:
idx = 100
instance = dataset[idx]

instance_dir = join(REPO_PATH, "figures", f"instance-{idx}")
os.makedirs(instance_dir, exist_ok=True)

for i, entry in enumerate(instance):
    del entry["full_caption"]

    visualize_entry_text_to_image(entry, pad_images=False, figsize=(19, 4))
    
    save_path = instance_dir
    plt.savefig(join(instance_dir, f"viz-{i}.png"), bbox_inches="tight", dpi=300)

<class 'numpy.ndarray'> 640 426


error: OpenCV(4.5.3) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src data type = 23 is not supported
>  - Expected Ptr<cv::UMat> for argument 'src'


In [ ]:
def create_and_save_gif(filenames, save_path, **kwargs):
    import imageio
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave(save_path, images, **kwargs)

In [ ]:
filenames = natsorted(glob(join(instance_dir, "viz-*.png")))
# save_path = join(instance_dir, "together.gif")
save_path = join(REPO_PATH, "media", "sample.gif")

create_and_save_gif(filenames, save_path, duration=3)

## Debugging code for a single entry

In [ ]:
idx = 100
instance = dataset[idx]
entry = instance[1]
del entry["full_caption"]

In [ ]:
entry.keys()

In [ ]:
visualize_item(**entry)

### Visualize everything together

In [ ]:
test_img, test_texts, orig_image = process_entry_text_to_image(entry, unimodal=False)
outputs = interpret_and_generate(model, test_img, test_texts, orig_image, return_outputs=True, show=False)
relevance_map = outputs[0]["image_relevance"]

In [ ]:
image_with_mask = overlay_segmask_on_image(entry["image"], entry["image_mask"])
image_with_mask = pad_to_square(image_with_mask)

In [ ]:
image_with_mask.size

In [ ]:
relevance_map.min(), relevance_map.max(), relevance_map.shape

In [ ]:
image_with_relevance_map = overlay_relevance_map_on_image(entry["image"], relevance_map)
image_with_relevance_map = pad_to_square(image_with_relevance_map)

In [ ]:
text_colors = get_text_colors(entry["text"], entry["text_mask"])

In [ ]:
show_images_and_caption([image_with_mask, image_with_relevance_map], entry["text"], text_colors, figsize=(18, 5))
plt.savefig("../figures/viz-1.pdf", bbox_inches="tight")